# OpenAI Assistant agent

In [1]:
# ! pip3 install openai

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
from agentquest.benchmarks.mastermind import MasterMindDriver, MasterMindUtils

game = MasterMindUtils.load_data(data_path="__default__", category="4 digits")[2]
game

'9605'

Initialize the prompt and the OpenAI Assistant agent

In [4]:
from openai import OpenAI

Initialize the mastermind driver and get the first observation

In [5]:
driver = MasterMindDriver(game)
obs = driver.reset()  # Get the first observation
print(f"OBSERVATION: {obs.output}")

OBSERVATION: You are tasked to play the Mastermind game.
The host chooses a number and gives you the amount of digits. You have to guess the correct number as fast as you can.
The number can contain repetitions and any possible digit between: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9.
At each round, you provide a number as a guess. At each step, the host provides you this information:
1. The number of correct digits in the wrong position.
2. The number of correct digits in the correct position.
The game ends when the host outputs 'You Won!'
Carefully choose your strategy. Avoid brute force.
The guess must be in the following format:
Guess: <number>
Start guessing the 4 digits number.


In [6]:
client = OpenAI()

assistant = client.beta.assistants.create(
    name="Mastermind Numbers Game Playing Agent",
    instructions="Your goal is to play mastermind number game following the instructions provided.",
    tools=[],
    model="gpt-4o",
)

In [7]:
print(assistant.id)

asst_HuwMROOeetYJax5CWR9eGRrs


Start the Agent loop solving the task

In [8]:
obs

MasterMindObservation(output="You are tasked to play the Mastermind game.\nThe host chooses a number and gives you the amount of digits. You have to guess the correct number as fast as you can.\nThe number can contain repetitions and any possible digit between: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9.\nAt each round, you provide a number as a guess. At each step, the host provides you this information:\n1. The number of correct digits in the wrong position.\n2. The number of correct digits in the correct position.\nThe game ends when the host outputs 'You Won!'\nCarefully choose your strategy. Avoid brute force.\nThe guess must be in the following format:\nGuess: <number>\nStart guessing the 4 digits number.", success=False, can_proceed=True)

In [9]:
thread = client.beta.threads.create()
print(thread.id)

# message = client.beta.threads.messages.create(
#     thread_id=thread.id,
#     role="user",
#     content=obs.output
# )

step_cnt = 0

while not obs.success and obs.can_proceed and step_cnt < 5:
    run = client.beta.threads.runs.create_and_poll(
        thread_id=thread.id, assistant_id=assistant.id, instructions=obs.output
    )
    print("Agent is running...")
    while run.status != "completed":
        pass
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    agent_output = list(messages)[0].content[0].text.value
    print("Agent output: ", agent_output)
    obs = driver.step_raw(agent_output)
    print(obs.output)
    step_cnt += 1

thread_sDZSmLyrtUgzHl3IR1rPpADT
Agent is running...
Agent output:  Let's begin playing the Mastermind game for a 4-digit number.

For my first guess, I'll try to get some initial information about which digits might be in the correct number. A common strategy is to start with repeated digits, such as:

Guess: 1111

Please let me know the feedback: number of correct digits in wrong positions, and number of correct digits in correct positions.
Wrong! Your guess has 0 correct digit in the correct position and 0 correct digit in the wrong position. Keep guessing.
Agent is running...
Agent output:  You're correct! The digit '1' is not in the secret number at all. Let's try a different guess:

Guess: 2222

Please provide feedback on this guess.
Wrong! Your guess has 0 correct digit in the correct position and 0 correct digit in the wrong position. Keep guessing.
Agent is running...
Agent output:  Let’s try another guess with a different digit:

Guess: 3456

Please provide feedback on this gu

In [13]:
driver.metrics.export(
    repetition_function_kwargs={"theta_a": 1, "num_execution_steps": 5}
)

{'goal': '9605',
 'success': False,
 'actions': [{'value': '1111'},
  {'value': '2222'},
  {'value': '3456'},
  {'value': '5634'},
  {'value': '2546'}],
 'states': [{'value': '1111'},
  {'value': '2222'},
  {'value': '3456'},
  {'value': '5634'},
  {'value': '2546'}],
 'observations': [{'output': 'Wrong! Your guess has 0 correct digit in the correct position and 0 correct digit in the wrong position. Keep guessing.',
   'success': False,
   'can_proceed': True},
  {'output': 'Wrong! Your guess has 0 correct digit in the correct position and 0 correct digit in the wrong position. Keep guessing.',
   'success': False,
   'can_proceed': True},
  {'output': 'Wrong! Your guess has 0 correct digit in the correct position and 2 correct digits in the wrong positions. Keep guessing.',
   'success': False,
   'can_proceed': True},
  {'output': 'Wrong! Your guess has 1 correct digit in the correct position and 1 correct digit in the wrong position. Keep guessing.',
   'success': False,
   'can_pr